In [1]:
import os
import json
import pandas as pd
from dataclasses import asdict

from datetime import datetime
from utils.config import DRLConfig
from utils.drl_train import training_pipeline

%load_ext autoreload
%autoreload 2

In [2]:
# SnP500 Sector
# DATA_DIR = "../data/snp_new"
# RETURNS_PATH = os.path.join(DATA_DIR, "returns_1d.parquet")
# PRICES_PATH = os.path.join(DATA_DIR, "prices_1d.parquet")
# VOLA_PATH = os.path.join(DATA_DIR, "vola_1d.parquet")

# MSCI World Index
DATA_DIR = "../data/intl"
RETURNS_PATH = os.path.join(DATA_DIR, "returns_1d.parquet")
PRICES_PATH = os.path.join(DATA_DIR, "prices_1d.parquet")
VOLA_PATH = os.path.join(DATA_DIR, "vola_1d.parquet")

df_ret = pd.read_parquet(RETURNS_PATH)
df_prices = pd.read_parquet(PRICES_PATH)
df_vol = pd.read_parquet(VOLA_PATH)

In [3]:
# To view the logs:
# 1. Open a terminal or command prompt.
# 2. Navigate to the directory *containing* the `logs` directory (i.e., the root of this repository).
# 3. Run the command: `tensorboard --logdir logs/`
# 4. Open the URL provided by TensorBoard (usually http://localhost:6006/) in your web browser.

# Create timestamp for this run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

N_WINDOWS = 10  # 10 in paper
N_AGENTS = 5  # 5 in paper
START_YR = 2006  # 2006 in paper
TOTAL_STEPS = 3_500_000 # 7_500_000 in paper
SEED_POLICY = True
FOLDER_NAME = f"{timestamp}_train_start={START_YR}_best_seed={SEED_POLICY}"

# Create configuration
config = DRLConfig(
    # Window configuration
    n_windows=N_WINDOWS,
    agents_per_window=N_AGENTS,
    base_start_year=START_YR,
    seed_policy=SEED_POLICY,
    # Environment parameters
    env_window_size=60,
    transaction_cost=0.0,
    initial_balance=100_000,
    reward_scaling=1.0,
    eta_dsr=1 / 252,
    # Training parameters
    n_envs=10,
    total_timesteps_per_round=TOTAL_STEPS,
    n_steps_per_env=252 * 3,
    batch_size=1260,
    n_epochs=16,
    gamma=0.9,
    gae_lambda=0.9,
    clip_range=0.25,
    log_std_init=-1.0,
    # Learning rate parameters
    initial_lr=3e-4,
    final_lr=1e-5,
    # Paths
    data_dir=DATA_DIR,
    model_save_dir=f"../models/{FOLDER_NAME}",
    tensorboard_log_dir=f"../logs/{FOLDER_NAME}",
    # prev_best_model_dir="../models/full_random_run/agent_6-1_seed=25_test=2017_valrew=43.20.zip",
)

config_dict = asdict(config)
config_json_path = os.path.join(config.model_save_dir, f"config_{timestamp}.json")
os.makedirs(config.model_save_dir, exist_ok=True)
with open(config_json_path, "w") as f:
    json.dump(config_dict, f, indent=4)
print(f"\nConfiguration saved to: {config_json_path}")


Configuration saved to: ../models/20250702_131032_train_start=2006_best_seed=True/config_20250702_131032.json


In [ ]:
# Run training pipeline
results_df, backtest_portfolio = training_pipeline(
    drl_config=config, df_prices=df_prices, df_ret=df_ret, df_vol=df_vol
)

--- Starting Window 1/10 (Train Year Start: 2006) ---
  Train Period: 2006-01-01 to 2010-12-31
  Val Period  : 2011-01-01 to 2011-12-31
  Test Period : 2012-01-01 to 2012-12-31
  Starting with fresh random initialization
  Training Agent 1/5 with seed 60924...


Output()


Training complete. Trained for 3500000 timesteps.
TensorBoard logs for experiment 'PPO_Seed=60924' saved in directory: ../logs/20250702_131032_train_start=2006_best_seed=True
    Evaluating agent on validation set...
    Validation Reward: -15.05362840
    Agent saved to: ../models/20250702_131032_train_start=2006_best_seed=True/agent_1-1_seed=60924_test=2012_valrew=-15.05.zip

  Training Agent 2/5 with seed 38161...


Output()


Training complete. Trained for 3500000 timesteps.
TensorBoard logs for experiment 'PPO_Seed=38161' saved in directory: ../logs/20250702_131032_train_start=2006_best_seed=True
    Evaluating agent on validation set...
    Validation Reward: -14.92597279
    Agent saved to: ../models/20250702_131032_train_start=2006_best_seed=True/agent_1-2_seed=38161_test=2012_valrew=-14.93.zip

  Training Agent 3/5 with seed 26185...


Output()

In [ ]:
print("Final Results DataFrame:")
results_df